### Custom metric for overfitting detector and best model selection

#### 1. Create an object that implements the following interface:

In [1]:
class CustomMetric(object):
    def get_final_error(self, error, weight):
        return 0.0

    def is_max_optimal(self):
        return True

    def evaluate(self, approxes, target, weight):
        # approxes - list of list-like objects (one object per approx dimension)
        # target - list-like object
        # weight - list-like object, can be None
        return 0.0, 0.0

#### The following is an example of the Logloss function implementation:

In [2]:
import math
from six.moves import xrange
from catboost import Pool, CatBoostClassifier

class LoglossMetric(object):
    def get_final_error(self, error, weight):
        return error / (weight + 1e-38)

    def is_max_optimal(self):
        return True

    def evaluate(self, approxes, target, weight):
        # approxes is list of indexed containers
        # (containers with only __len__ and __getitem__ defined), one container
        # per approx dimension. Each container contains floats.
        # weight is one dimensional indexed container.
        # target is float.   
        # weight parameter can be None.
        # Returns pair (error, weights sum)

        assert len(approxes) == 1
        assert len(target) == len(approxes[0])

        approx = approxes[0]

        error_sum = 0.0
        weight_sum = 0.0

        for i in xrange(len(approx)):
            w = 1.0 if weight is None else weight[i]
            weight_sum += w
            error_sum += w * (target[i] * approx[i] - math.log(1 + math.exp(approx[i])))

        return error_sum, weight_sum

cat_features = [0, 1, 2]

train_data = [["a", "b", 1, 4, 5, 6],
              ["a", "b", 4, 5, 6, 7],
              ["c", "d", 30, 40, 50, 60]]

train_labels = [1, 1, 0]

eval_data = [["a", "c", 3, 4, 4, 1],
             ["a", "d", 1, 5, 5, 5],
             ["b", "d", 31, 25, 60, 70]]

eval_labels = [0, 1, 1]

eval_dataset = Pool(eval_data,
                    label=eval_labels,
                    cat_features=cat_features)
# Initialize CatBoostClassifier with custom `eval_metric`
model = CatBoostClassifier(iterations=5,
                           eval_metric=LoglossMetric())
# Fit model with `use_best_model=True`
model.fit(train_data,
          train_labels,
          cat_features,
          use_best_model=True,
          eval_set=eval_dataset)

# Get predictions
pred = model.predict(eval_data)

Learning rate set to 0.40944
0:	learn: -0.6449595	test: -0.7049507	best: -0.7049507 (0)	total: 52.4ms	remaining: 210ms
1:	learn: -0.6169377	test: -0.7262057	best: -0.7049507 (0)	total: 54.7ms	remaining: 82.1ms
2:	learn: -0.5763070	test: -0.7113233	best: -0.7049507 (0)	total: 56.2ms	remaining: 37.5ms
3:	learn: -0.5397138	test: -0.7492843	best: -0.7049507 (0)	total: 57.6ms	remaining: 14.4ms
4:	learn: -0.5066801	test: -0.7868770	best: -0.7049507 (0)	total: 58.9ms	remaining: 0us

bestTest = -0.7049507077
bestIteration = 0

Shrink model to first 1 iterations.


#### 2. Pass the created object to the eval_metric parameter:

CatBoostClassifier(eval_metric=CustomMetric())

### Custom objective(loss) function

A custom objective function can be used by specifying a python object as the value for the loss_function parameter. In this case the objective is always maximized.

Depending on the machine learning problem the python object should have one the following functions defined:
* calc_ders_range
* calc_ders_multi(approxes, target, weight) (for multiclassification)

In [5]:
import math
from six.moves import xrange
from catboost import Pool, CatBoostClassifier


class LoglossObjective(object):
    def calc_ders_range(self, approxes, targets, weights):
        # approxes, targets, weights are indexed containers of floats
        # (containers with only __len__ and __getitem__ defined).
        # weights parameter can be None.
        # Returns list of pairs (der1, der2)
        assert len(approxes) == len(targets)
        if weights is not None:
            assert len(weights) == len(approxes)

        exponents = []
        for index in xrange(len(approxes)):
            exponents.append(math.exp(approxes[index]))

        result = []
        for index in xrange(len(targets)):
            p = exponents[index] / (1 + exponents[index])
            der1 = (1 - p) if targets[index] > 0.0 else -p
            der2 = -p * (1 - p)

            if weights is not None:
                der1 *= weights[index]
                der2 *= weights[index]

            result.append((der1, der2))

        return result

cat_features = [0, 1]

train_data = [[1, 4, 5, 6],
              [4, 5, 6, 7],
              [30, 40, 50, 60]]

train_labels = [1, 1, 0]

eval_data = [[3, 4, 4, 1],
             [1, 5, 5, 5],
             [31, 25, 60, 70]]

# Initialize CatBoostClassifier with custom `loss_function`
model = CatBoostClassifier(loss_function=LoglossObjective(),
                           eval_metric="Logloss")
# Fit model
model.fit(train_data, train_labels)
# Only prediction_type='RawFormulVal' allowed with custom `loss_function`
preds_raw = model.predict(eval_data,
                          prediction_type='RawFormulaVal')

print(preds_raw)

0:	learn: 0.6908422	total: 3.21ms	remaining: 3.21s
1:	learn: 0.6885477	total: 6.06ms	remaining: 3.03s
2:	learn: 0.6849617	total: 7.57ms	remaining: 2.52s
3:	learn: 0.6826944	total: 9.46ms	remaining: 2.36s
4:	learn: 0.6791523	total: 10.9ms	remaining: 2.17s
5:	learn: 0.6756375	total: 12.3ms	remaining: 2.04s
6:	learn: 0.6734131	total: 13.6ms	remaining: 1.93s
7:	learn: 0.6711990	total: 15.2ms	remaining: 1.89s
8:	learn: 0.6677432	total: 16.8ms	remaining: 1.85s
9:	learn: 0.6643140	total: 18.3ms	remaining: 1.81s
10:	learn: 0.6609111	total: 19.8ms	remaining: 1.78s
11:	learn: 0.6587544	total: 21.4ms	remaining: 1.76s
12:	learn: 0.6553930	total: 22.6ms	remaining: 1.72s
13:	learn: 0.6520574	total: 24.1ms	remaining: 1.7s
14:	learn: 0.6499413	total: 25.6ms	remaining: 1.68s
15:	learn: 0.6478350	total: 27ms	remaining: 1.66s
16:	learn: 0.6457384	total: 28.5ms	remaining: 1.65s
17:	learn: 0.6436513	total: 29.6ms	remaining: 1.62s
18:	learn: 0.6404013	total: 31ms	remaining: 1.6s
19:	learn: 0.6371760	total: 

281:	learn: 0.3011743	total: 323ms	remaining: 823ms
282:	learn: 0.3005388	total: 324ms	remaining: 822ms
283:	learn: 0.2999056	total: 327ms	remaining: 824ms
284:	learn: 0.2992746	total: 328ms	remaining: 822ms
285:	learn: 0.2986458	total: 329ms	remaining: 821ms
286:	learn: 0.2980193	total: 330ms	remaining: 820ms
287:	learn: 0.2973949	total: 331ms	remaining: 818ms
288:	learn: 0.2967728	total: 332ms	remaining: 817ms
289:	learn: 0.2961529	total: 333ms	remaining: 816ms
290:	learn: 0.2955351	total: 334ms	remaining: 814ms
291:	learn: 0.2949195	total: 335ms	remaining: 813ms
292:	learn: 0.2943061	total: 336ms	remaining: 811ms
293:	learn: 0.2936948	total: 337ms	remaining: 810ms
294:	learn: 0.2930857	total: 338ms	remaining: 809ms
295:	learn: 0.2924786	total: 340ms	remaining: 808ms
296:	learn: 0.2918737	total: 341ms	remaining: 806ms
297:	learn: 0.2912710	total: 342ms	remaining: 805ms
298:	learn: 0.2906703	total: 343ms	remaining: 804ms
299:	learn: 0.2900717	total: 344ms	remaining: 802ms
300:	learn: 

591:	learn: 0.1761963	total: 665ms	remaining: 458ms
592:	learn: 0.1759466	total: 666ms	remaining: 457ms
593:	learn: 0.1756976	total: 667ms	remaining: 456ms
594:	learn: 0.1754492	total: 668ms	remaining: 455ms
595:	learn: 0.1752014	total: 669ms	remaining: 454ms
596:	learn: 0.1749543	total: 670ms	remaining: 452ms
597:	learn: 0.1747077	total: 671ms	remaining: 451ms
598:	learn: 0.1744618	total: 672ms	remaining: 450ms
599:	learn: 0.1742166	total: 673ms	remaining: 449ms
600:	learn: 0.1739719	total: 674ms	remaining: 448ms
601:	learn: 0.1737279	total: 675ms	remaining: 447ms
602:	learn: 0.1734845	total: 676ms	remaining: 445ms
603:	learn: 0.1732417	total: 677ms	remaining: 444ms
604:	learn: 0.1729995	total: 679ms	remaining: 443ms
605:	learn: 0.1727579	total: 680ms	remaining: 442ms
606:	learn: 0.1725169	total: 681ms	remaining: 441ms
607:	learn: 0.1722765	total: 682ms	remaining: 440ms
608:	learn: 0.1720367	total: 683ms	remaining: 438ms
609:	learn: 0.1717976	total: 684ms	remaining: 437ms
610:	learn: 

895:	learn: 0.1217806	total: 983ms	remaining: 114ms
896:	learn: 0.1216534	total: 985ms	remaining: 113ms
897:	learn: 0.1215265	total: 986ms	remaining: 112ms
898:	learn: 0.1213999	total: 987ms	remaining: 111ms
899:	learn: 0.1212735	total: 988ms	remaining: 110ms
900:	learn: 0.1211473	total: 989ms	remaining: 109ms
901:	learn: 0.1210214	total: 990ms	remaining: 108ms
902:	learn: 0.1208957	total: 991ms	remaining: 106ms
903:	learn: 0.1207703	total: 992ms	remaining: 105ms
904:	learn: 0.1206451	total: 993ms	remaining: 104ms
905:	learn: 0.1205201	total: 994ms	remaining: 103ms
906:	learn: 0.1203954	total: 995ms	remaining: 102ms
907:	learn: 0.1202709	total: 996ms	remaining: 101ms
908:	learn: 0.1201467	total: 996ms	remaining: 99.8ms
909:	learn: 0.1200227	total: 997ms	remaining: 98.6ms
910:	learn: 0.1198990	total: 998ms	remaining: 97.5ms
911:	learn: 0.1197754	total: 999ms	remaining: 96.4ms
912:	learn: 0.1196521	total: 1s	remaining: 95.3ms
913:	learn: 0.1195291	total: 1s	remaining: 94.2ms
914:	learn: 